In [14]:
import pandas as pd
import json
import numpy as np

In [2]:
df = pd.read_csv('PA_test_task2_JD.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159896 entries, 0 to 2159895
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   event_time        object
 2   event_type        object
 3   event_properties  object
dtypes: object(4)
memory usage: 65.9+ MB


Спочатку я хочу поділити дані, аби було зручніше робити аналітику. 

я вже проглянув попередньо датасет і бачу, що дві групи виділяються на основі 

experiment - started == {"variation_id":"0"} 

experiment - started == {"variation_id":"1"} 

In [ ]:
exp = df.loc[df["event_type"] == "experiment - started", ["user_id", "event_time", "event_properties"]].copy()

#парсинг JSON
exp["event_properties"] = exp["event_properties"].fillna("{}")

def safe_json_loads(x):
    if isinstance(x, dict):
        return x
    if not isinstance(x, str) or x.strip() == "":
        return {}
    try:
        return json.loads(x)
    except Exception:
        return {}

props = exp["event_properties"].apply(safe_json_loads)
exp["variation_id"] = props.apply(lambda d: str(d.get("variation_id")) if d.get("variation_id") is not None else None)

exp["event_time"] = pd.to_datetime(exp["event_time"], utc=True, errors="coerce")
exp = exp.dropna(subset=["user_id", "event_time", "variation_id"])

#беремо перще призначення для кожного юзера
assign = (
    exp.sort_values("event_time")
       .groupby("user_id", as_index=False)
       .first()[["user_id", "variation_id"]]
)

#мапим в test/control
assign["ab_group"] = assign["variation_id"].map({"0": "control", "1": "test"})


df = df.merge(assign[["user_id", "ab_group"]], on="user_id", how="left")

#check:

#івенти
print(df["ab_group"].value_counts(dropna=False))

#спліт по користувачах
print(df.drop_duplicates("user_id")["ab_group"].value_counts())
print(df.drop_duplicates("user_id")["ab_group"].value_counts(normalize=True))

ab_group
control    1277292
test        881437
NaN           1167
Name: count, dtype: int64
ab_group
control    38342
test       16563
Name: count, dtype: int64
ab_group
control    0.698333
test       0.301667
Name: proportion, dtype: float64


тут є NaN = 1167, але це досить мала частина від даних(близько 0.05%), тому ми можемо це викинути. 


In [5]:
df_ab = df.dropna(subset=["ab_group"]).copy()

тепер зручно бачити статус користувачів

Базовою нуль-гіпотезою я виділив: Додавання туторіалу не збільшило рівень задоволеності відповідями (к-сть лайків відповіді). Водночас я би хотів оцінити інші фактори і, можливо, створити кращу гіпотезу. По-перше, варто подивитись на к-сть користувачів, які пройшли туторіал, також яка к-сть користувачів із тестової вибірки залишає фідбек загалом vs така к-сть з контрольної вибірки. Також варто оцінити adoption rate самого туторіалу(тобто чи хтось клікає на перше вікно чи проходить загалом). 

Загалом, мій хід думок полягав в тому, на що може вплинути наявність ви відсутність туторіалу на платформі(особливо з точки зору людини, яка майже ніколи не проходить туторіали, а витрачає білььше часу, аби розібратись самому.)

Як на мене, туторіал може впливати на задоволеність роботою тулзи(або можна сказати, що задоволеність результатом роботи), адже якщо користувач яркза розуміє логіку фічі, то краще можу користуватись і вдповідно не мати хибних очікувань, а задоволеність якраз вибудовується якщо очікування і реальність збігаються, retention(вже як рещультат задоволеності). 

В контексті оцінки цих факторів я думав над тим, чи можлива більша задоволеність у тестовій вибірці не може бути пов'язана із тим, що користувачі, які пройшли туторіал будуть більш вмотивовані і відповідно більш задоволені роботою тулки, тому в гіпотезі варто оцінювати результати усієї вибірки, а не лише ту частку користувачів, які пройшли туторіал. 

Тому в підсумку:

H0: Додавання інтерактивного туторіалу не підвищує рівень задоволеності результатами роботи тулів.


H1: Додавання інтерактивного туторіалу підвищує рівень задоволеності результатами роботи тулів.

In [ ]:
# мій перший інстинкт - це було подивитись на абсолютну к-сть лайків у двох різних групах та знайти співвідношення

like_dislike_counts = (
    df_ab[df_ab["event_type"].isin(["response - click like", "response - click dislike"])]
    .groupby(["ab_group", "event_type"])
    .size()
    .unstack(fill_value=0)
)

like_dislike_counts["total_reactions"] = (
    like_dislike_counts["response - click like"] +
    like_dislike_counts["response - click dislike"]
)

like_dislike_counts["like_share"] = (
    like_dislike_counts["response - click like"] / like_dislike_counts["total_reactions"]
)

like_dislike_counts["dislike_share"] = (
    like_dislike_counts["response - click dislike"] / like_dislike_counts["total_reactions"]
)

like_dislike_counts

event_type,response - click dislike,response - click like,total_reactions,like_share,dislike_share
ab_group,,,,,
control,1817,2751,4568,0.602233,0.397767
test,683,1198,1881,0.636895,0.363105


але мені здається, що це можуть бути трішки misleading результати, адже тут ми рахуємо співвідношення між абсолютною к-стю лайків та дизлайків у кожній групі і це значить, що надто активні позитивні/негативні юзери можуть змістити наші значення, тому я хочу спробувати агрегувати ці значення на базі кожного юзера і подивитись, який буде результат тоді. 

In [15]:
reactions = df_ab[df_ab["event_type"].isin(["response - click like", "response - click dislike"])].copy()
reactions["cnt"] = 1

user_reactions = (
    reactions.pivot_table(
        index=["user_id", "ab_group"],
        columns="event_type",
        values="cnt",
        aggfunc="sum",
        fill_value=0
    )
    .reset_index()
    .rename(columns={
        "response - click like": "likes",
        "response - click dislike": "dislikes"
    })
)

user_reactions["total"] = user_reactions["likes"] + user_reactions["dislikes"]
user_reactions["satisfaction_rate"] = user_reactions["likes"] / user_reactions["total"]

group_satisfaction_mean = user_reactions.groupby("ab_group")["satisfaction_rate"].mean()
group_satisfaction_mean

ab_group
control    0.630423
test       0.652679
Name: satisfaction_rate, dtype: float64

Ми вже бачимо певні результати, але тепер треба красиво це виконати і створити більш формальний A/B тест. 

In [16]:
primary_summary = (
    user_reactions.groupby("ab_group")["satisfaction_rate"]
    .agg(
        users_reacted="count",
        mean="mean",
        median="median",
        p25=lambda x: x.quantile(0.25),
        p75=lambda x: x.quantile(0.75),
    )
)

primary_summary

,users_reacted,mean,median,p25,p75
ab_group,,,,,
control,3498,0.630423,1.0,0.0,1.0
test,1476,0.652679,1.0,0.0,1.0


In [17]:
control_mean = primary_summary.loc["control", "mean"]
test_mean = primary_summary.loc["test", "mean"]

uplift_abs = test_mean - control_mean
uplift_rel = uplift_abs / control_mean

print(f"Control mean satisfaction: {control_mean:.4f}")
print(f"Test mean satisfaction:    {test_mean:.4f}")
print(f"Absolute uplift:           {uplift_abs:.4f} ({uplift_abs*100:.2f} pp)")
print(f"Relative uplift:           {uplift_rel*100:.2f}%")


Control mean satisfaction: 0.6304
Test mean satisfaction:    0.6527
Absolute uplift:           0.0223 (2.23 pp)
Relative uplift:           3.53%


Отож, результат ідеї порахувати метрику задоволеності для кожного юзера:

- Test mean satisfaction > Control mean satisfaction
- uplift = 2.23 (та Relative uplift: 3.53%)

Це значить, що в тестовій групі користувачі (які залишають реакції) частіше оцінюють результат позитивно, ніж у контрольній.

In [18]:
#хто користувався тулками
tool_users = (
    df_ab[df_ab["event_type"] == "tools - send message"][["user_id", "ab_group"]]
    .drop_duplicates()
)

#користувачі, які задлишили реакцію
react_users = (
    df_ab[df_ab["event_type"].isin(["response - click like", "response - click dislike"])][["user_id", "ab_group"]]
    .drop_duplicates()
)

feedback_rate = (
    tool_users.groupby("ab_group")["user_id"].nunique()
    .rename("tool_users")
    .to_frame()
    .join(react_users.groupby("ab_group")["user_id"].nunique().rename("react_users"))
)

feedback_rate["feedback_rate"] = feedback_rate["react_users"] / feedback_rate["tool_users"]
feedback_rate

,tool_users,react_users,feedback_rate
ab_group,,,
control,37305,3498,0.093768
test,16123,1476,0.091546


Окремо я порахував feedback_rate, щоб зрозуміти чи туторіал впливає на поведінку “залишати фідбек”, а не тільки на satisfaction.

primary metric я рахую тільки серед тих, хто реагує. Якщо feedback_rate різний між групами, то можливий bias.

Далі я порівняю feedback_rate між control і test, щоб підтвердити, що порівнювати satisfaction правильно.

In [22]:
test_users_viewed_tool = (
    df_ab[
        (df_ab["ab_group"] == "test") &
        (df_ab["event_type"].str.contains(r"view.*tool", case=False, na=False))
    ][["user_id"]]
    .drop_duplicates()
)

test_users_tut_started = (
    df_ab[(df_ab["ab_group"] == "test") & (df_ab["event_type"] == "tutorial - started")][["user_id"]]
    .drop_duplicates()
)

test_users_tut_finished = (
    df_ab[(df_ab["ab_group"] == "test") & (df_ab["event_type"] == "tutorial - finished")][["user_id"]]
    .drop_duplicates()
)

n_viewed = len(test_users_viewed_tool)
n_started = len(test_users_tut_started)
n_finished = len(test_users_tut_finished)

tutorial_adoption = pd.DataFrame([{
    "test_users_viewed_tool": n_viewed,
    "test_users_tut_started": n_started,
    "test_users_tut_finished": n_finished,
    "tutorial_start_rate": (n_started / n_viewed) if n_viewed else np.nan,
    "tutorial_finish_rate_among_started": (n_finished / n_started) if n_started else np.nan,
    "tutorial_finish_rate_among_viewed": (n_finished / n_viewed) if n_viewed else np.nan,
}])

tutorial_adoption

,test_users_viewed_tool,test_users_tut_started,test_users_tut_finished,tutorial_start_rate,tutorial_finish_rate_among_started,tutorial_finish_rate_among_viewed
0,16315,13334,12991,0.817285,0.974276,0.796261


Також я хотів зрозуміти чи фіча реально використовується.

Тому в adoption Test я дивився на tutorial_start_rate та tutorial_finish_rate

Якщо adoption низький, то навіть корисний туторіал може показати слабкий ефект.

In [23]:
#аctivation(user-level): view tool -> send message

view_users = (
    df_ab[
        df_ab["event_type"].str.contains(r"view.*tool", case=False, na=False)
    ][["user_id", "ab_group"]]
    .drop_duplicates()
)

send_users = (
    df_ab[df_ab["event_type"] == "tools - send message"][["user_id", "ab_group"]]
    .drop_duplicates()
)

activation = (
    view_users.groupby("ab_group")["user_id"].nunique()
    .rename("view_users")
    .to_frame()
    .join(send_users.groupby("ab_group")["user_id"].nunique().rename("send_users"))
)

activation["activation_rate"] = activation["send_users"] / activation["view_users"]
activation

,view_users,send_users,activation_rate
ab_group,,,
control,37720,37305,0.988998
test,16315,16123,0.988232


Туторіал має допомогти користувачам краще зрозуміти функції тулів, додатково оцінив activation:

Ця метрика показує, чи допоміг туторіал користувачам перейти від перегляду тули до реального використання (початку генерації). Бачимо, що рещультати практично ідентичні для обох вибірок, тому можемо сказати, що туторіал майже не вплинува на активацію.

In [ ]:
def bootstrap_diff_mean(a, b, n_boot=10000, seed=42):
    rng = np.random.default_rng(seed)
    a = np.asarray(a); b = np.asarray(b)
    obs = b.mean() - a.mean()

    diffs = np.empty(n_boot)
    for i in range(n_boot):
        diffs[i] = rng.choice(b, size=len(b), replace=True).mean() - rng.choice(a, size=len(a), replace=True).mean()

    ci_low, ci_high = np.quantile(diffs, [0.025, 0.975])
    p_one_sided = (diffs <= 0).mean()
    return obs, (ci_low, ci_high), p_one_sided

control = user_reactions[user_reactions["ab_group"] == "control"]["satisfaction_rate"].values
test = user_reactions[user_reactions["ab_group"] == "test"]["satisfaction_rate"].values

obs_diff, ci, p_one_sided = bootstrap_diff_mean(control, test)

print("Primary metric: mean user-level satisfaction_rate among reacting users")
print(f"Diff (Test - Control): {obs_diff:.4f}")
print(f"95% bootstrap CI: [{ci[0]:.4f}, {ci[1]:.4f}]")
print(f"One-sided p-value (H1: Test > Control): {p_one_sided:.4f}")

Primary metric: mean user-level satisfaction_rate among reacting users
Diff (Test - Control): 0.0223
95% bootstrap CI: [-0.0058, 0.0504]
One-sided p-value (H1: Test > Control): 0.0601



Я хотів перевірити, чи uplift не є випадковим шумом, томв використав bootstrap тест.

Результати bootstrap:

Diff (Test − Control) = +0.0223, тобто у тесті satisfaction вища на +2.23 pp (позитивний напрям)

95% CI = [-0.0058; 0.0504], інтервал включає 0, тобто ми не можемо виключити, що реальний ефект = 0 або навіть трохи негативний

p-value (one-sided) = 0.0601, це  >0.05, тому на рівні значущості 5% ми не відхиляємо H0

Загалом, у Test satisfaction вище на 2.23 pp., але ефект не досяг статистичної значущості (p=0.0601), а 95% CI перетинає 0. Тому на основі цих даних нема достатньо доказів, що туторіал реально підвищує satisfaction. Також, p-value близький до порогу 0.05, тому результат можна трактувати як слабкий сигнал на користь позитивного ефекту, але доказів недостатньо, аби робити повний і  впевнений rollout.

Рекомендація:

1) Не робити 100% rollout лише на основі цього


2) оптимізувати туторіал, аби збільшити adoption

3) додати secondary metrics у decision: activation / feedback_rate
